In [1]:
from mdp_sequence_reader import MDPSequenceReader
from training_pomdp_util import *
from observable_markov_model import ObservableMarkovModel

In [2]:
# read the sequence
reader = MDPSequenceReader('sample_sequence/o_0_0.txt')
print("Num states: %d" % reader.get_num_states())
print("Num steps: %d" % reader.get_num_steps())

Num states: 8
Num steps: 10000


In [3]:
# get the sequences
observations = reader.get_observation_sequence()
actions = reader.get_action_sequence()

In [4]:
# get the initial model
action_space = [0, 1, 2, 3]   # (up, right, down, left)
num_states = reader.get_num_states()
num_observables = 16   # binary measurements of 4 directions

init_model = initialize_random_pomdp_model(num_states, action_space, num_observables)

In [5]:
model = init_model
for round in range(1):
    alist, c = improve_params(xs=actions, ys=observations, m=model)
    new_model = ObservableMarkovModel(alist, c, model.init)
    
    log_likelihood = get_log_likelihood(make_tableaus(xs=actions, ys=observations, m=new_model))
    print("round=%d, log likelihood=%f" % (round+1, log_likelihood))
    
    model = new_model

round=1, log likelihood=-16557.603321


In [6]:
print(alist, c)

{0: array([[0.11389152, 0.14600324, 0.07649407, 0.03568036, 0.15935937,
        0.00877359, 0.07239832, 0.02174907],
       [0.2568761 , 0.18229823, 0.0296935 , 0.09862883, 0.13573773,
        0.12798683, 0.06366273, 0.09801948],
       [0.06349383, 0.05261104, 0.18483067, 0.00608877, 0.26992184,
        0.26100401, 0.03683327, 0.33473252],
       [0.09959572, 0.21102117, 0.16737209, 0.05514354, 0.06909017,
        0.02336184, 0.00723726, 0.15236579],
       [0.13400934, 0.07676451, 0.04928519, 0.02196934, 0.10712585,
        0.12056909, 0.41284895, 0.35061804],
       [0.08505875, 0.20602255, 0.07288553, 0.64045299, 0.11335451,
        0.17129076, 0.23119388, 0.01733907],
       [0.05412086, 0.02054886, 0.16548349, 0.06914946, 0.00834576,
        0.21275216, 0.04754354, 0.00391991],
       [0.19295389, 0.10473041, 0.25395548, 0.07288671, 0.13706477,
        0.07426173, 0.12828204, 0.02125612]]), 1: array([[0.02975129, 0.03699294, 0.06152515, 0.00856377, 0.05125944,
        0.01603475,